In [1]:
from astropy.io import fits
import astropy.units as u
from astropy.constants import G
from astropy.table import Table

import numpy as np

In [2]:
ic_data, header = fits.getdata("data/outputs/perseus_ic348_only.fits",header=True)
ngc_data, header = fits.getdata("data/outputs/perseus_ngc1333_only.fits",header=True)

In [3]:
stats = Table.read("data/outputs/deconvolution_results.csv", format = "csv")

## Mass Calculations

In [4]:
# voxel to mass conversion
def get_mass(data):
    return np.sum(data * (1.37 * (1.6726231 * (10**(-24))) * 2.938 * (10 ** (55)))) * (5.02785 * (10**(-34)))

In [5]:
ic_mass = get_mass(ic_data)
ngc_mass = get_mass(ngc_data)

In [30]:
print(r"$M$")

$M$


In [31]:
print(f"IC 348 Gas Mass {ic_mass} M_sun")
print(f"NGC 1333 Gas Mass {ngc_mass} M_sun")

IC 348 Gas Mass 5.911511327850987 M_sun
NGC 1333 Gas Mass 37.576070601517706 M_sun


## Dispersion Calculations

In [8]:
def predicted_dispersion(M, rhm) :
    n = 10 #see Cottaar et al. 2015
    return np.sqrt((M*G)/(n * rhm)).to(u.km / u.s)

def predicted_3ddispersion(M, rhm) :
    n = 10 #see Cottaar et al. 2015
    return np.sqrt((3*M*G)/(n * rhm)).to(u.km / u.s)

In [9]:
def real_dispersion(idx): #idx == 0 for IC 348; idx ==1 for NGC 1333
    return stats["UVW_dispersion"][idx]

##### IC 348

In [10]:
ic_M_stellar = 204 * u.solMass # see Cottaar et al. 2015
ic_M_gas = ic_mass * u.solMass
ic_rhm = 0.47 * u.pc # see Cottaar et al. 2015
ic_corrected = 6.3/60 * (np.pi/180)*stats['D'][0] * u.pc
ic_rhm = stats['L_err'][0] * (np.pi/180)*stats['D'][0] * u.pc # calculated using ellipse width and distance

ic_M = ic_M_stellar.to(u.kg) + ic_M_gas.to(u.kg)
ic_R = ic_rhm.to(u.m)

In [21]:
ic_real3d = real_dispersion(0)
ic_pred3d = predicted_3ddispersion(ic_M, ic_R)
ic_pred1d = predicted_dispersion(ic_M, ic_R)

##### NGC 1333

In [16]:
ngc_M_stellar = 102 * u.solMass # see Foster et al. 2015
ngc_M_gas = ngc_mass * u.solMass
# ngc_rhm = 0.97 * u. pc # see Foster et al. 2015
ngc_rhm = stats['L_err'][1] * (np.pi/180)*stats['D'][1] * u.pc # calculated using ellipse width and distance

ngc_M = ngc_M_stellar.to(u.kg) + ngc_M_gas.to(u.kg)
ngc_R = ngc_rhm.to(u.m)

In [22]:
ngc_real3d = real_dispersion(1)
ngc_pred3d = predicted_3ddispersion(ngc_M, ngc_R)
ngc_pred1d = predicted_dispersion(ngc_M, ngc_R)

In [20]:
print(f"NGC 1333 real 3D:      {ngc_real3d}")
print(f"NGC 1333 Predicted 3D: {ngc_pred3d}")

print(f"IC 348 real 3D:        {ic_real3d}")
print(f"IC 348 Predicted 3D:   {ic_pred3d}")

NGC 1333 real 3D:      2.0572921870377283
NGC 1333 Predicted 3D: 0.8307029471723544 km / s
IC 348 real 3D:        1.874728250551058
IC 348 Predicted 3D:   0.761267783197558 km / s


## Bounded-ness

In [ ]:
# r_vel deconvolution:
# IC 348: 0.67956875
# NGC 1333: 0.99079429

In [23]:
ngc_pred1d

<Quantity 0.47960657 km / s>

In [24]:
ic_pred1d

<Quantity 0.43951816 km / s>